In [1]:
from __future__ import print_function, division
import numpy as np
import sympy.physics.mechanics as me
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)
import sympy as sp
from sympy import symbols,diff,Symbol
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,Particle, Point)

In [2]:
#I'm writing modified form functions to express the position of the end effector
#using Denavit-Hartenberg matrices


def Rotation(axis,angle):
    Matrix=0        
    if axis=='x':
        Matrix=sp.Matrix([[1,0,0,0],
                          [0,sp.cos(angle),-sp.sin(angle),0],
                          [0,sp.sin(angle),sp.cos(angle),0],
                          [0,0,0,1]])
    
    elif axis=='y':
        Matrix=sp.Matrix([[sp.cos(angle),0,sp.sin(angle),0],
                          [0,1,0,0],
                          [-sp.sin(angle),0,sp.cos(angle),0],
                          [0,0,0,1]])
        
    elif axis=='z':
        Matrix=sp.Matrix([[sp.cos(angle),-sp.sin(angle),0,0],
                          [sp.sin(angle),sp.cos(angle),0,0],
                          [0,0,1,0],
                          [0,0,0,1]])
    return(Matrix)

def Displace(vector):
    Matrix=sp.Matrix([[1,0,0,vector[0]],
                      [0,1,0,vector[1]],
                      [0,0,1,vector[2]],
                      [0,0,0,1]])
    return(Matrix)

def Parallel(Inertia,mass,r):
    """Inertia matrix, mass value, r vector (position)"""
    skew=skew_symmetric(r)
    Matrix=Inertia+mass*(skew.transpose()*skew)
    return(sp.simplify(Matrix))
                       
def skew_symmetric(vector):
    matrix=sp.Matrix([[0,-vector[2],vector[1]],
                     [vector[2],0,-vector[0]],
                     [-vector[1],vector[0],0]])
    return(matrix)
    

def angular_velocity_inertial(Matrix):
    Rotation_Matrix=Matrix[0:3,0:3]
    matrix=diff(Rotation_Matrix,Symbol('t'))*Rotation_Matrix.transpose()
    display(diff(Rotation_Matrix,Symbol('t')))
    display(Rotation_Matrix.transpose())
    display(sp.simplify(matrix))
    vector=sp.Matrix([-matrix[1,2],matrix[0,2],-matrix[0,1]])
    return(sp.simplify(vector))

def Jacobian(Matrix,variables):
    vector=Matrix[0:3,3]
    variables=sp.Matrix([variables])
    return(vector.jacobian(variables))
    

In [3]:
l1,l2=symbols('l_1,l2')
theta1,theta2,h=dynamicsymbols('theta1,theta2,h')
dtheta1,dtheta2,dh=sp.diff(theta1,Symbol('t')),sp.diff(theta2,Symbol('t')),sp.diff(h,Symbol('t'))
ddtheta1,ddtheta2,ddh=sp.diff(dtheta1,Symbol('t')),sp.diff(dtheta2,Symbol('t')),sp.diff(dh,Symbol('t'))


In [4]:
P=Rotation('z',theta1)*Displace([l1,0,0])*Displace([0,h,0])*Rotation('z',theta2)*Displace([0,l2,0])

#General function to obtain the acceleration of the end effector (or any other known point in the arm)
aP=sp.simplify(diff(P,Symbol('t'),2))[0:3,3]

#Generation of function to evaluate the symbolic functions
aP_lamb=sp.lambdify(([l1,l2],
                     [theta1,h,theta2],
                     [dtheta1,dh,dtheta2],
                     [ddtheta1,ddh,ddtheta2]),aP)

aP

Matrix([
[ l2*(sin(theta1 + theta2)*theta1'**2 + 2*sin(theta1 + theta2)*theta1'*theta2' + sin(theta1 + theta2)*theta2'**2 - cos(theta1 + theta2)*theta1'' - cos(theta1 + theta2)*theta2'') - l_1*sin(theta1)*theta1'' - l_1*cos(theta1)*theta1'**2 + h*sin(theta1)*theta1'**2 - h*cos(theta1)*theta1'' - sin(theta1)*h'' - 2*cos(theta1)*h'*theta1'],
[-l2*(sin(theta1 + theta2)*theta1'' + sin(theta1 + theta2)*theta2'' + cos(theta1 + theta2)*theta1'**2 + 2*cos(theta1 + theta2)*theta1'*theta2' + cos(theta1 + theta2)*theta2'**2) - l_1*sin(theta1)*theta1'**2 + l_1*cos(theta1)*theta1'' - h*sin(theta1)*theta1'' - h*cos(theta1)*theta1'**2 - 2*sin(theta1)*h'*theta1' + cos(theta1)*h''],
[                                                                                                                                                                                                                                                                                                                                    

In [5]:
P=Rotation('z',theta1)*Displace([1,0,0])#*Displace([0,h,0])*Rotation('z',theta2)*Displace([0,l2,0])

#General function to obtain the acceleration of the end effector (or any other known point in the arm)
aP=sp.simplify(diff(P,Symbol('t'),2))[0:3,3]
display(aP)

P=Rotation('z',theta1)*Displace([1,0,0])*Displace([0,h,0])#*Rotation('z',theta2)*Displace([0,l2,0])

#General function to obtain the acceleration of the end effector (or any other known point in the arm)
aP=sp.simplify(diff(P,Symbol('t'),2))[0:3,3]
display(aP)

P=Rotation('z',theta1)*Displace([1,0,0])*Displace([0,h,0])*Rotation('z',theta2)*Displace([0,1,0])

#General function to obtain the acceleration of the end effector (or any other known point in the arm)
aP=sp.simplify(diff(P,Symbol('t'),2))[0:3,3]
display(aP)

Matrix([
[-sin(theta1)*theta1'' - cos(theta1)*theta1'**2],
[-sin(theta1)*theta1'**2 + cos(theta1)*theta1''],
[                                             0]])

Matrix([
[ h*sin(theta1)*theta1'**2 - h*cos(theta1)*theta1'' - sin(theta1)*h'' - sin(theta1)*theta1'' - 2*cos(theta1)*h'*theta1' - cos(theta1)*theta1'**2],
[-h*sin(theta1)*theta1'' - h*cos(theta1)*theta1'**2 - 2*sin(theta1)*h'*theta1' - sin(theta1)*theta1'**2 + cos(theta1)*h'' + cos(theta1)*theta1''],
[                                                                                                                                              0]])

Matrix([
[ h*sin(theta1)*theta1'**2 - h*cos(theta1)*theta1'' + sin(theta1 + theta2)*theta1'**2 + 2*sin(theta1 + theta2)*theta1'*theta2' + sin(theta1 + theta2)*theta2'**2 - sin(theta1)*h'' - sin(theta1)*theta1'' - cos(theta1 + theta2)*theta1'' - cos(theta1 + theta2)*theta2'' - 2*cos(theta1)*h'*theta1' - cos(theta1)*theta1'**2],
[-h*sin(theta1)*theta1'' - h*cos(theta1)*theta1'**2 - sin(theta1 + theta2)*theta1'' - sin(theta1 + theta2)*theta2'' - 2*sin(theta1)*h'*theta1' - sin(theta1)*theta1'**2 - cos(theta1 + theta2)*theta1'**2 - 2*cos(theta1 + theta2)*theta1'*theta2' - cos(theta1 + theta2)*theta2'**2 + cos(theta1)*h'' + cos(theta1)*theta1''],
[                                                                                                                                                                                                                                                                                                                           0]])

In [6]:
                  """[l1,l2],
                     [theta1,h,theta2],
                     [dtheta1,dh,dtheta2],
                     [ddtheta1,ddh,ddtheta2]"""

#accelerations of the center of mass of link 1
a_C1=sp.Matrix(aP_lamb([0.5,0],
                  [0,0,0],
                  [dtheta1,0,0],
                  [ddtheta1,0,0]))

#accelerations of the center of mass of link 2
a_C2=sp.Matrix(aP_lamb([1,0],
                  [0,h/2,0],
                  [dtheta1,dh,0],
                  [ddtheta1,ddh,0]))

#accelerations of the center of mass of link 3
a_C3=sp.Matrix(aP_lamb([1,0.5],
                  [0,h,0],
                  [dtheta1,dh,dtheta2],
                  [ddtheta1,ddh,ddtheta2]))

#masses
m1,m2,m3=1,0.5,1

F3=m3*a_C3
F2=m2*a_C2+m3*a_C3
F1=m1*a_C1+m2*a_C2+m3*a_C3

display('F3: ',F3,'F2: ',F2,'F1: ',F1)

'F3: '

Matrix([
[                   -1.0*h*theta1'' - 2.0*h'*theta1' - 1.0*theta1'**2 - 0.5*theta1'' - 0.5*theta2''],
[-1.0*h*theta1'**2 + 1.0*h'' - 0.5*theta1'**2 - 1.0*theta1'*theta2' + 1.0*theta1'' - 0.5*theta2'**2],
[                                                                                                 0]])

'F2: '

Matrix([
[                   -1.25*h*theta1'' - 3.0*h'*theta1' - 1.5*theta1'**2 - 0.5*theta1'' - 0.5*theta2''],
[-1.25*h*theta1'**2 + 1.5*h'' - 0.5*theta1'**2 - 1.0*theta1'*theta2' + 1.5*theta1'' - 0.5*theta2'**2],
[                                                                                                  0]])

'F1: '

Matrix([
[                   -1.25*h*theta1'' - 3.0*h'*theta1' - 2.0*theta1'**2 - 0.5*theta1'' - 0.5*theta2''],
[-1.25*h*theta1'**2 + 1.5*h'' - 0.5*theta1'**2 - 1.0*theta1'*theta2' + 2.0*theta1'' - 0.5*theta2'**2],
[                                                                                                  0]])

In [7]:
I1,I2,I3=0.1,0.05,0.1

#Vectors of angular velocities
w1,w2,w3=sp.Matrix([[0],[0],[dtheta1]]),sp.Matrix([[0],[0],[dtheta1]]),sp.Matrix([[0],[0],[dtheta1+dtheta2]])

#Vector of angular accelerations
dw1,dw2,dw3=sp.diff(w1,Symbol('t')),sp.diff(w2,Symbol('t')),sp.diff(w3,Symbol('t'))

#Vector of position r_(i,ci) (from next joint to center of mass)
r1,r2,r3=sp.Matrix([[-0.5],[0],[0]]),sp.Matrix([[0],[-h/2],[0]]),sp.Matrix([[0],[-0.5],[0]])

#Vector of position r_(i-1,ci) (From previous joint to center of mass)
r1_,r2_,r3_=sp.Matrix([[0.5],[0],[0]]),sp.Matrix([[0],[h/2],[0]]),sp.Matrix([[0],[0.5],[0]])

Tau3=I3*dw3+w3.cross(I3*w3)-(F3.cross(r3_))

Tau2=I2*dw2+w2.cross(I2*w2)-(F2.cross(r2_)-F3.cross(r3))+Tau3

Tau1=I1*dw1+w1.cross(I1*w1)-(F1.cross(r1_)-F2.cross(r2))+Tau2

display('Tau3: ',Tau3,'Tau2: ',Tau2,'Tau1: ',Tau1)


'Tau3: '

Matrix([
[                                                                               0],
[                                                                               0],
[0.5*h*theta1'' + 1.0*h'*theta1' + 0.5*theta1'**2 + 0.35*theta1'' + 0.35*theta2'']])

'Tau2: '

Matrix([
[                                                                                                                                                                        0],
[                                                                                                                                                                        0],
[-(-1.25*h*theta1'' - 3.0*h'*theta1' - 1.5*theta1'**2 - 0.5*theta1'' - 0.5*theta2'')*h/2 + 1.0*h*theta1'' + 2.0*h'*theta1' + 1.0*theta1'**2 + 0.65*theta1'' + 0.6*theta2'']])

'Tau1: '

Matrix([
[                                                                                                                                                                                                                                               0],
[                                                                                                                                                                                                                                               0],
[-(-1.25*h*theta1'' - 3.0*h'*theta1' - 1.5*theta1'**2 - 0.5*theta1'' - 0.5*theta2'')*h - 0.625*h*theta1'**2 + 1.0*h*theta1'' + 2.0*h'*theta1' + 0.75*h'' + 0.75*theta1'**2 - 0.5*theta1'*theta2' + 1.75*theta1'' - 0.25*theta2'**2 + 0.6*theta2'']])